# Get Data

In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader

stock_name = "A"
df = pd.read_csv(
    "stocks/{}.csv".format(stock_name),
    index_col="Date",
    parse_dates=True
)

train_size = int(0.67 * df.index.size)
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(df)

X = []
y = []

num_window = 20
idx = 0

for idx in range(len(data)-num_window):
    X.append(data[idx:idx+num_window,:])
    y.append(data[idx+num_window,:])
    
X = torch.tensor(np.array(X), requires_grad=True, dtype=torch.float64).view(-1, num_window, df.columns.size)
y = torch.tensor(np.array(y), requires_grad=True, dtype=torch.float64).view(-1, 1, df.columns.size)

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [2]:
class Data(Dataset):
    def __init__(self, x, y):
        super(Data, self).__init__()
        self.x = x 
        self.y = y
        self.len = x.shape[0]
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    def __len__(self):
        return self.len


In [3]:
data_train = Data(X_train, y_train)
data_test = Data(X_test, y_test)

train_loader = DataLoader(data_train, batch_size=100)
test_loader = DataLoader(data_test, batch_size=100)

# Define Model

In [32]:
from torch import nn, optim

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, out_features):
        super(LSTMModel, self).__init__()
        self.h0 = torch.zeros(num_layers, X_train.size(1),  hidden_size, dtype=torch.float64)
        self.c0 = torch.zeros(num_layers, X_train.size(1),  hidden_size, dtype=torch.float64)

        self.lstm = nn.LSTM(
            input_size=input_size
            , hidden_size=hidden_size
            , num_layers=num_layers
            , dtype=torch.float64
            , bias=True
        )
        self.linear = nn.Linear(
            in_features=hidden_size
            , out_features=out_features
            , dtype=torch.float64
        )


    def forward(self, activation):
        activation, (hn, cn) = self.lstm(activation, (self.h0.detach(), self.c0.detach()))
        activation = self.linear(activation)
        activation = activation[:, -1, :].view(-1, 1, out_features)
        return activation#, (hn, cn)
    
input_size=X_train.size(-1)
hidden_size=16
num_layers=1
out_features=6
model = LSTMModel(input_size, hidden_size, num_layers, out_features)

# Train Test Loop

In [33]:
learning_rate = 0.001
num_epochs = 100
criterion = nn.MSELoss()
optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate
)

In [34]:
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 10 != 0:
        continue
    model.eval()
    with torch.no_grad():
        y_pred = model(X_train)
        train_rmse = np.sqrt(criterion(y_pred, y_train))
        y_pred = model(X_test)
        test_rmse = np.sqrt(criterion(y_pred, y_test))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

Epoch 0: train RMSE 0.0811, test RMSE 0.2175
Epoch 10: train RMSE 0.0245, test RMSE 0.0315
Epoch 20: train RMSE 0.0228, test RMSE 0.0322
Epoch 30: train RMSE 0.0196, test RMSE 0.0264
Epoch 40: train RMSE 0.0188, test RMSE 0.0251
Epoch 50: train RMSE 0.0182, test RMSE 0.0210
Epoch 60: train RMSE 0.0181, test RMSE 0.0218
Epoch 70: train RMSE 0.0200, test RMSE 0.0458
Epoch 80: train RMSE 0.0219, test RMSE 0.0248
Epoch 90: train RMSE 0.0189, test RMSE 0.0260


In [31]:
scaler.inverse_transform(model(X_train[0].view(1, -1, 6))[0].detach().numpy())[0][0]

28.54562683636425

In [18]:
scaler.inverse_transform(y_train[0].detach().numpy())[0][0]

33.17238998413086

In [ ]:
import matplotlib.pyplot as plt 